<a href="https://colab.research.google.com/github/francejules22/Type2DiabetesPredictionsModel/blob/main/Type2DiabetesFinalModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Step 1: Importing The Libraries and Dependencies

In [31]:
%matplotlib inline 
import pandas as pd # Data processing, Input & Output load 
import numpy as np  # linear algebra 
import seaborn as sns
import matplotlib.pyplot as plt
#from sklearn.preprocessing import LabelEncoder #performs the conversion of these labels of categorical data into a numeric format.
#from sklearn.preprocessing import StandardScaler #standardizing data such the the transformed feature has 0 mean and and a standard deviation of 1.

#Import the model
import xgboost as xgb
from sklearn.ensemble import GradientBoostingClassifier    # GBM algorithm
from xgboost.sklearn import XGBClassifier    # Extreme Gradient Boosting


import joblib  #Joblib is a set of tools to provide lightweight pipelining in Python (Avoid computing twice the same thing / save the model)
from sklearn.model_selection import train_test_split   # train_test_split - Split arrays or matrices into random train and test subsets
                                    
                                    
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score                               


import warnings    # To avoid warning messages in the code run
warnings.filterwarnings('ignore')

## Step 2: Load the Dataset

In [32]:
diabetes = pd.read_csv('TypeIIDiabetes.csv')    
diabetes

,Age,Gender,Polyuria,Polydipsia,sudden weight loss,weakness,Polyphagia,Genital thrush,visual blurring,Itching,Irritability,delayed healing,partial paresis,muscle stiffness,Alopecia,Obesity,class
0,40,Male,No,Yes,No,Yes,No,No,No,Yes,No,Yes,No,Yes,Yes,Yes,Positive
1,58,Male,No,No,No,Yes,No,No,Yes,No,No,No,Yes,No,Yes,No,Positive
2,41,Male,Yes,No,No,Yes,Yes,No,No,Yes,No,Yes,No,Yes,Yes,No,Positive
3,45,Male,No,No,Yes,Yes,Yes,Yes,No,Yes,No,Yes,No,No,No,No,Positive
4,60,Male,Yes,Yes,Yes,Yes,Yes,No,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Positive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
515,39,Female,Yes,Yes,Yes,No,Yes,No,No,Yes,No,Yes,Yes,No,No,No,Positive
516,48,Female,Yes,Yes,Yes,Yes,Yes,No,No,Yes,Yes,Yes,Yes,No,No,No,Positive
517,58,Female,Yes,Yes,Yes,Yes,Yes,No,Yes,No,No,No,Yes,Yes,No,Yes,Positive
518,32,Female,No,No,No,Yes,No,No,Yes,Yes,No,Yes,No,No,Yes,No,Negative


## Step 3: Applying the Label Encoder

In [33]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

for i in diabetes.columns[1:] :
    diabetes[i] = le.fit_transform(diabetes[i])

diabetes.head() 

,Age,Gender,Polyuria,Polydipsia,sudden weight loss,weakness,Polyphagia,Genital thrush,visual blurring,Itching,Irritability,delayed healing,partial paresis,muscle stiffness,Alopecia,Obesity,class
0,40,1,0,1,0,1,0,0,0,1,0,1,0,1,1,1,1
1,58,1,0,0,0,1,0,0,1,0,0,0,1,0,1,0,1
2,41,1,1,0,0,1,1,0,0,1,0,1,0,1,1,0,1
3,45,1,0,0,1,1,1,1,0,1,0,1,0,0,0,0,1
4,60,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1


## Step 4: Spliting Independent and dependent variables

a.) Indepentent Variables

In [34]:
x=diabetes.iloc[:,:-1].values
x

array([[40,  1,  0, ...,  1,  1,  1],
       [58,  1,  0, ...,  0,  1,  0],
       [41,  1,  1, ...,  1,  1,  0],
       ...,
       [58,  0,  1, ...,  1,  0,  1],
       [32,  0,  0, ...,  0,  1,  0],
       [42,  1,  0, ...,  0,  0,  0]])

b.) Dependent Variables

In [35]:
y=diabetes.iloc[:,-1].values
y

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1,

## Step 5: Transform Data using Standard Scaler

In [36]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x=sc.fit_transform(x)
x

array([[-0.6613669 ,  0.76509206, -0.99233705, ...,  1.29099445,
         1.38022749,  2.21564684],
       [ 0.82136224,  0.76509206, -0.99233705, ..., -0.77459667,
         1.38022749, -0.45133547],
       [-0.57899306,  0.76509206,  1.00772212, ...,  1.29099445,
         1.38022749, -0.45133547],
       ...,
       [ 0.82136224, -1.30703226,  1.00772212, ...,  1.29099445,
        -0.72451824,  2.21564684],
       [-1.32035762, -1.30703226, -0.99233705, ..., -0.77459667,
         1.38022749, -0.45133547],
       [-0.49661921,  0.76509206, -0.99233705, ..., -0.77459667,
        -0.72451824, -0.45133547]])

## Step 6: Save the transform data using joblib

In [37]:
import joblib  
joblib.dump(sc,'diabetes_transform')

['diabetes_transform']

## Step 7: Splitting dataset into train and test

In [38]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(x,y,test_size=0.2,random_state=0)

print('Train Shape: ', X_train.shape)
print('Test Shape: ', Y_test.shape)

Train Shape:  (416, 16)
Test Shape:  (104,)


## Step 8: Construct Model using Gradient Boosting 

In [39]:
gradient_booster = GradientBoostingClassifier(learning_rate=0.1)
gradient_booster.get_params()

{'ccp_alpha': 0.0,
 'criterion': 'friedman_mse',
 'init': None,
 'learning_rate': 0.1,
 'loss': 'deviance',
 'max_depth': 3,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_iter_no_change': None,
 'random_state': None,
 'subsample': 1.0,
 'tol': 0.0001,
 'validation_fraction': 0.1,
 'verbose': 0,
 'warm_start': False}

### a.) Refitting the data

In [40]:
gradient_booster.fit(X_train, Y_train)

GradientBoostingClassifier()

### b.) Displaying model prediction and classification report

In [42]:
train_pred = gradient_booster.predict(X_train)
test_pred = gradient_booster.predict(X_test)

In [43]:
print('Classification report for train data is : \n',
      classification_report(Y_train, train_pred))
print('Classification report for test data is : \n',
      classification_report(Y_test, test_pred))

Classification report for train data is : 
               precision    recall  f1-score   support

           0       1.00      0.99      1.00       160
           1       1.00      1.00      1.00       256

    accuracy                           1.00       416
   macro avg       1.00      1.00      1.00       416
weighted avg       1.00      1.00      1.00       416

Classification report for test data is : 
               precision    recall  f1-score   support

           0       0.98      1.00      0.99        40
           1       1.00      0.98      0.99        64

    accuracy                           0.99       104
   macro avg       0.99      0.99      0.99       104
weighted avg       0.99      0.99      0.99       104



### c.) Making predictions for test data by testing the model

In [47]:
y_pred = gradient_booster.predict(X_test)
predictions = [round(value) for value in y_pred]
predictions

[1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1]

### d.) Checking the prediction accuracy for test data of the model

In [49]:
accuracy = accuracy_score(Y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 99.04%


### e.) Saving the best model

In [50]:
joblib.dump(gradient_booster, 'diabetes.save')

['diabetes.save']

### Step 9: Testing if it really works 

#### a.) Positive Patient

1. patient #1: 
age = 40, gender=1 , polyuria=0 , polydipsia=1, sudden_weight_loss =0, weakness=1 , polyphagia=0, genital_thrush=0, visual_blurring =0 , itching=1, irritability=0, delayed_healing=1, partial_paresis=0, muscle_stiffness=1, alopecia=1, obesity=1, class=positive/1 

In [56]:
print(gradient_booster.predict([[40,1,0,1,0,1,0,0,0,1,0,1,0,1,1,1]]))

[1]


b.) Negative Patient

2.) Patient #504: age=70, gender=0 , polyuria=1, polydipsia=0, sudden_weight_loss =0, weakness=0, polyphagia=1, genital_thrush=0, visual_blurring=1 , itching=1, irritability=0, delayed_healing=1, partial_paresis=1, muscle_stiffness=1, alopecia=1, obesity=0, class=negative/0

In [57]:
print(gradient_booster.predict([[70,0,1,0,0,0,1,0,1,1,0,1,1,1,1,0]]))

[0]
